In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyomo.environ import *
from random import shuffle
from pyomo.util import model_size
from random import sample
%run plotting.ipynb  
np.random.seed(0)

In [2]:
def create_model(data):
    
    # unpacking data
    nrakes = len(data['a'])
    ndest = data['pendency'].shape[0]
    nterm = data['pendency'].shape[1]
    BigM = data['BigM']
    BigM2 = data['BigM2']
    
    M = ConcreteModel()
    
    # Adding variables
    M.Sa = Var(RangeSet(nrakes),RangeSet(len(data['sys'])),domain=NonNegativeReals)
    M.Sb = Var(RangeSet(nrakes),RangeSet(len(data['sys'])),domain=NonNegativeReals) 
    M.xa = Var(RangeSet(int(nrakes*(nrakes-1)/2)),RangeSet(len(data['sys'])),domain=Binary)
    M.xb = Var(RangeSet(int(nrakes*(nrakes-1)/2)),RangeSet(len(data['sys'])),domain=Binary)  
    M.w1a = Var(RangeSet(nrakes),RangeSet(data['sys'][0]),domain=Binary)
    M.w2a = Var(RangeSet(nrakes),RangeSet(data['sys'][1]),domain=Binary)
    M.w4a = Var(RangeSet(nrakes),RangeSet(sum(data['sys'][3])),domain=Binary)
    M.w1b = Var(RangeSet(nrakes),RangeSet(data['sys'][0]),domain=Binary)
    M.w2b = Var(RangeSet(nrakes),RangeSet(data['sys'][1]),domain=Binary)
    M.y = Var(RangeSet(nrakes),RangeSet(nterm),domain=Binary)
    M.v = Var(RangeSet(nrakes),RangeSet(ndest),domain=Binary)
    M.g = Var(RangeSet(nrakes),RangeSet(nterm),domain=NonNegativeIntegers,bounds=(0,data['R'][1]))
    M.n = Var(RangeSet(nrakes),RangeSet(ndest),RangeSet(nterm),domain=NonNegativeIntegers,bounds=(0,data['R'][1]))
    M.t4 = Var(RangeSet(nrakes),domain=NonNegativeReals)
    M.z = Var(RangeSet(nrakes),domain=Binary)
    
    # Adding constraints
    
    M.seqa_cons = ConstraintList() # 1
    for i in range(1,nrakes+1):
        t1 = data['t1'][data['origin'][i-1]-1]
        M.seqa_cons.add(expr=M.Sa[i,1]+t1<=M.Sa[i,2])
        M.seqa_cons.add(expr=M.Sa[i,2]+data['t2']<=M.Sa[i,3])
        M.seqa_cons.add(expr=M.Sa[i,3]+data['t3']<=M.Sa[i,4])
    
    M.seqb_cons = ConstraintList() # 2,3,4
    for i in range(1,nrakes+1):
        M.seqb_cons.add(expr=M.Sb[i,4] >= M.Sa[i,4]+M.t4[i])
        M.seqb_cons.add(expr=M.Sb[i,3] >= M.Sb[i,4])
        M.seqb_cons.add(expr=M.Sb[i,2] >= M.Sb[i,3]+data['t3'])
        M.seqb_cons.add(expr=M.Sb[i,1] >= M.Sb[i,2]+data['t2'])
            
    M.scheda_cons = ConstraintList() # 5,6
    idx = 1
    for i in range(1,nrakes):
        for i1 in range(i+1,nrakes+1):
            for m in range(1,data['sys'][0]+1):
                M.scheda_cons.add(expr=M.Sa[i,1] >= M.Sa[i1,2]-BigM*(1-M.xa[idx,1])-BigM*(2-M.w1a[i,m]-M.w1a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,1] >= M.Sa[i,2]-BigM*M.xa[idx,1]-BigM*(2-M.w1a[i,m]-M.w1a[i1,m]))
            for m in range(1,data['sys'][1]+1):
                M.scheda_cons.add(expr=M.Sa[i,2] >= M.Sa[i1,3]-BigM*(1-M.xa[idx,2])-BigM*(2-M.w2a[i,m]-M.w2a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,2] >= M.Sa[i,3]-BigM*M.xa[idx,2]-BigM*(2-M.w2a[i,m]-M.w2a[i1,m]))
            for m in range(1,data['sys'][2]+1):
                M.scheda_cons.add(expr=M.Sa[i,3] >= M.Sa[i1,3]-BigM*(1-M.xa[idx,3])-BigM*(2-M.y[i,m]-M.y[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,3] >= M.Sa[i,3]-BigM*M.xa[idx,3]-BigM*(2-M.y[i,m]-M.y[i1,m]))
            for m in range(1,sum(data['sys'][3])+1):
                M.scheda_cons.add(expr=M.Sa[i,4] >= M.Sb[i1,3]-BigM*(1-M.xa[idx,4])-BigM*(2-M.w4a[i,m]-M.w4a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,4] >= M.Sb[i,3]-BigM*M.xa[idx,4]-BigM*(2-M.w4a[i,m]-M.w4a[i1,m]))            
            idx += 1
    
    M.schedb_cons = ConstraintList() # 7,8
    idx = 1
    for i in range(1,nrakes):
        for i1 in range(i+1,nrakes+1):
            for m in range(1,data['sys'][2]+1):
                M.schedb_cons.add(expr=M.Sb[i,3] >= M.Sb[i1,2]-BigM*(1-M.xb[idx,3])-BigM*(2-M.y[i,m]-M.y[i1,m]))
                M.schedb_cons.add(expr=M.Sb[i1,3] >= M.Sb[i,2]-BigM*(M.xb[idx,3])-BigM*(2-M.y[i,m]-M.y[i1,m]))
            for m in range(1,data['sys'][1]+1):
                M.schedb_cons.add(expr=M.Sb[i,2] >= M.Sb[i1,1]-BigM*(1-M.xb[idx,2])-BigM*(2-M.w2b[i,m]-M.w2b[i1,m]))
                M.schedb_cons.add(expr=M.Sb[i1,2] >= M.Sb[i,1]-BigM*(M.xb[idx,2])-BigM*(2-M.w2b[i,m]-M.w2b[i1,m]))
            for m in range(1,data['sys'][0]+1):
                t1 = data['t1'][data['origin'][i]-1]
                M.schedb_cons.add(expr=M.Sb[i,1] >= M.Sb[i1,1]+t1-BigM*(1-M.xb[idx,1])-BigM*(2-M.w1b[i,m]-M.w1b[i1,m]))
                M.schedb_cons.add(expr=M.Sb[i1,1] >= M.Sb[i,1]+t1-BigM*(M.xb[idx,1])-BigM*(2-M.w1b[i,m]-M.w1b[i1,m]))
            idx += 1              
    
    
           
    M.releases_cons = ConstraintList() # 9
    for i in range(1,nrakes+1):
        M.releases_cons.add(expr=M.Sa[i,1] >= data['a'][i-1])
    
    M.sumw1a_cons = ConstraintList() # 10.1
    for i in range(1,nrakes+1):
        M.sumw1a_cons.add(expr=sum(M.w1a[i,m] for m in range(1,data['sys'][0]+1))==1)
    
    M.sumw2a_cons = ConstraintList() # 10.2
    for i in range(1,nrakes+1):
        M.sumw2a_cons.add(expr=sum(M.w2a[i,m] for m in range(1,data['sys'][1]+1))==1)
        
    M.sumw4a_cons = ConstraintList() # 10.4
    for i in range(1,nrakes+1):
        M.sumw4a_cons.add(expr=sum(M.w4a[i,m] for m in range(1,sum(data['sys'][3])+1))==1)
    
    M.sumw1b_cons = ConstraintList() # 11.1
    for i in range(1,nrakes+1):
        M.sumw1b_cons.add(expr=sum(M.w1b[i,m] for m in range(1,data['sys'][0]+1))==1)
        
    M.sumw2b_cons = ConstraintList() # 11.2
    for i in range(1,nrakes+1):
        M.sumw2b_cons.add(expr=sum(M.w2b[i,m] for m in range(1,data['sys'][1]+1))==1)

    M.destlineassign_cons = ConstraintList() # 12
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            for m in range(1,data['sys'][0]+1):
                M.destlineassign_cons.add(expr=M.w1b[i,m] >= M.v[i,d]*data['l'][d-1,m-1])
                
    M.sumv_cons = ConstraintList() # 14
    for i in range(1,nrakes+1):
        M.sumv_cons.add(expr=sum(M.v[i,d] for d in range(1,ndest+1))==1)
    
    M.sumy_cons = ConstraintList() # 15
    for i in range(1,nrakes+1):
        M.sumy_cons.add(expr=sum(M.y[i,t] for t in range(1,nterm+1))==1)
        
    M.BPCdest_cons = ConstraintList() # 16
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            M.BPCdest_cons.add(expr=M.v[i,d] >= data['b'][i-1,d-1])
    
    M.minmaxload_cons = ConstraintList() # 17
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            M.minmaxload_cons.add(expr=data['R'][0] <= sum(M.n[i,d,t] for t in range(1,nterm+1)))
            M.minmaxload_cons.add(expr=data['R'][1] >= sum(M.n[i,d,t] for t in range(1,nterm+1)))

    M.load_cons = ConstraintList() # 18
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            for t in range(1,nterm+1):
                M.load_cons.add(expr=M.n[i,d,t] <= M.v[i,d]*data['pendency'][d-1,t-1])
    
    M.ittcompute_cons = ConstraintList() # 19,20(corrected), extra1
    for i in range(1,nrakes+1):
        for t in range(1,nterm+1):
            M.ittcompute_cons.add(expr=M.g[i,t] <= sum(M.n[i,d,t] for d in range(1,ndest+1))+BigM2*(1-M.y[i,t]))
            M.ittcompute_cons.add(expr=M.g[i,t] >= sum(M.n[i,d,t] for d in range(1,ndest+1))-BigM2*(1-M.y[i,t]))
            M.ittcompute_cons.add(expr=M.g[i,t] <= data['R'][1]*M.y[i,t])
#             M.ittcompute_cons.add(expr=M.g[i,t] <= data['R'][1]*M.v[i,d])
    
    M.origins_cons = ConstraintList() # extra2
    for i in range(1,nrakes+1):
        k = data['origin'][i-1]
        M.origins_cons.add(expr=M.w1a[i,k]==1)
    
    M.lastlinks_cons = ConstraintList() # extra3
    idx = 1
    for m1 in range(1,data['sys'][2]+1):
        for m2 in range(1,data['sys'][3][m1-1]+1):
            for i in range(1,nrakes+1):
                M.lastlinks_cons.add(expr=M.w4a[i,idx]<=M.y[i,m1])
            idx += 1
    
    M.sumpendency_cons = ConstraintList() # extra 4 (changed)
    for t in range(1,nterm+1):
        for d in range(1,ndest+1):
            M.sumpendency_cons.add(expr=sum(M.n[i,d,t] for i in range(1,nrakes+1)) <= data['pendency'][d-1,t-1])
    

    M.procTime_cons = ConstraintList() # extra, replace 13
    for i in range(1,nrakes+1):
        M.procTime_cons.add(expr=sum(M.g[i,t] for t in range(1,nterm+1)) <= data['Pure min limit']+BigM2*(1-M.z[i]))
        M.procTime_cons.add(expr=sum(M.g[i,t] for t in range(1,nterm+1)) >= data['Pure max limit']-BigM2*(M.z[i]))
        M.procTime_cons.add(expr=M.t4[i] == data['Loading times'][1]*M.z[i]+data['Loading times'][0]*(1-M.z[i]))
    
#     M.destwisesumn = ConstraintList()
#     for i in range(1,nrakes+1):
#         for d in range(2,3):
#             M.destwisesumn.add(expr=sum(M.n[i,d,t]for t in range(1,nterm+1))>=60*M.v[i,d])
#     M.destwisesumn.add(expr=sum(M.v[i,2] for i in range(1,nrakes+1))==1)
    
    M.obj = Objective(expr=data['c1']*(sum(M.Sb[i,1]-data['a'][i-1] for i in range(1,nrakes+1)))+data['cont cost']*(sum(sum(sum(M.n[i,d,t] for i in range(1,nrakes+1)) for d in range(1,ndest+1))for t in range(1,nterm+1))-data['pendency'].sum()),sense=maximize)  
    #M.obj = Objective(expr=0.0)
    
    return M


In [3]:
def get_resultdataframe(data,model1,fname='dummy_result'):
    nrakes = len(data['a'])
    ndest = data['pendency'].shape[0]
    nterm = data['pendency'].shape[1]
    BigM = data['BigM']
    BigM2 = data['BigM2']
    result_rows = []
    for i in range(1,nrakes+1):
        res1 = [data['a'][i-1]]+[max(0,model1.Sa[i,j].value) for j in range(1,5)]+[model1.Sb[i,4-j].value for j in range(0,4)]+[model1.t4[i].value]
        allocations = []
        l = []
        for m in range(1,data['sys'][0]+1):
            if model1.w1a[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 1",l)
        else:
            allocations.append(l[0])
        l = []
        for m in range(1,data['sys'][1]+1):
            if model1.w2a[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 2",l)
        else:
            allocations.append(l[0])
        l = []
        for m in range(1,sum(data['sys'][3])+1):
            if model1.w4a[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 2",l)
        else:
            allocations.append(l[0])
        l = []
        for m in range(1,data['sys'][1]+1):
            if model1.w2b[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 2",l)
        else:
            allocations.append(l[0])
        l = []
        for m in range(1,data['sys'][0]+1):
            if model1.w1b[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 1",l)
        else:
            allocations.append(l[0])
        l = []
        for d in range(1,ndest+1):
            if model1.v[i,d].value > 0.5:
                l.append(d)
        if len(l) > 1:
            print("Error 1",l)
        else:
            allocations.append(l[0])

        res1 += allocations

        res1.append(int(sum([model1.n[i,d,1].value for d in range(1,ndest+1)])))
        res1.append(int(sum([model1.n[i,d,2].value for d in range(1,ndest+1)])))
        l = []
        for m in range(1,data['sys'][2]+1):
            if model1.y[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 2",l)
        else:
            res1.append(l[0])
        for t in range(1,nterm+1):
            res1.append(int(model1.g[i,t].value))
        l = [model1.n[i,d,1].value for d in range(1,ndest+1)]
        rs = ['n-'+str(d)+'-'+str(1) for d in range(1,ndest+1)]
        for t in range(2,nterm+1):
            l += [model1.n[i,d,t].value for d in range(1,ndest+1)]
            rs += ['n-'+str(d)+'-'+str(t) for d in range(1,ndest+1)]
        res1+=l
        result_rows.append(res1)
    result_dataframe = pd.DataFrame(data=result_rows,columns=['Arr times','Sa 1','Sa 2','Sa 3','Sa 4','Sb 4','Sb 3','Sb 2','Sb 1','t4','alloca 1','alloca 2','alloca 4','allocb 2','allocb 1','dest','pend 1','pend 2','terminal allocated']+['g'+str(t) for t in range(1,nterm+1)]+rs)      
    result_dataframe.to_csv('data/'+fname+'.csv')
    return result_dataframe

In [4]:
def get_specTrains(result,m):
    r = result
    ans = []
    for i in r.index:
        for j in r.columns:
            if r.loc[i,j] >= m*24.0:
                ans.append(r.loc[i,:])
                break
    ans = pd.DataFrame(data=ans,columns=r.columns)
    return ans

In [5]:
def create_rollingModel(rolling_data,data,rolling):
    
    BigM = data['BigM']*2
    BigM2 = data['BigM2']
    
    M = create_model(data)
    if not rolling:
        return M
    cont_rakes = len(rolling_data.index)
    rolling_data.index = list(range(cont_rakes))
    nrakes = len(data['a'])
    M.cont_xa = Var(RangeSet(cont_rakes),RangeSet(nrakes),RangeSet(len(data['sys'])),domain=Binary)
    M.cont_xb = Var(RangeSet(cont_rakes),RangeSet(nrakes),RangeSet(len(data['sys'])),domain=Binary)
    w1a = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][0])]
    for i in range(cont_rakes):
        w1a[int(rolling_data.loc[i,'alloca 1']-1)][i] = 1
    w2a = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][1])]
    for i in range(cont_rakes):
        w2a[int(rolling_data.loc[i,'alloca 2']-1)][i] = 1
    y = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][2])]
    for i in range(cont_rakes):
        y[int(rolling_data.loc[i,'terminal allocated']-1)][i] = 1
    w4a = [[0 for _ in range(cont_rakes)]for _ in range(sum(data['sys'][3]))]
    for i in range(cont_rakes):
        w4a[int(rolling_data.loc[i,'alloca 4']-1)][i] = 1
    w1b = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][0])]
    for i in range(cont_rakes):
        w1b[int(rolling_data.loc[i,'allocb 1']-1)][i] = 1
    w2b = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][1])]
    for i in range(cont_rakes):
        w2b[int(rolling_data.loc[i,'allocb 2']-1)][i] = 1
    
    M.rolling_scheda_cons = ConstraintList()
    for i in range(1,nrakes+1):
        for i1 in range(1,cont_rakes+1):
            for m in range(1,data['sys'][0]+1):
                #M.rolling_scheda_cons.add(inequality(rolling_data.loc[i-1,'Sa 2']-BigM*(M.cont_xa[i1,i,1])-BigM*(2-M.w1a[i1,m]-w1a[m-1][i-1]),M.Sa[i1,1],rolling_data.loc[i-1,'Sa 1']+BigM*(1-M.cont_xa[i1,i,1])+BigM*(2-M.w1a[i1,m]-w1a[m-1][i-1])))
                M.rolling_scheda_cons.add(expr=rolling_data.loc[i1-1,'Sa 1'] >= M.Sa[i,1]-BigM*(1-M.cont_xa[i1,i,1])-BigM*(2-M.w1a[i,m]-w1a[m-1][i1-1]))
                M.rolling_scheda_cons.add(expr=M.Sa[i,1] >= rolling_data.loc[i1-1,'Sa 2']-BigM*(M.cont_xa[i1,i,1])-BigM*(2-M.w1a[i,m]-w1a[m-1][i1-1]))    
            for m in range(1,data['sys'][1]+1):
                #M.rolling_scheda_cons.add(inequality(rolling_data.loc[i-1,'Sa 3']-BigM*(M.cont_xa[i1,i,2])-BigM*(2-M.w2a[i1,m]-w2a[m-1][i-1]),M.Sa[i1,2],rolling_data.loc[i-1,'Sa 2']+BigM*(1-M.cont_xa[i1,i,2])+BigM*(2-M.w2a[i1,m]-w2a[m-1][i-1])))
                M.rolling_scheda_cons.add(expr=rolling_data.loc[i1-1,'Sa 2'] >= M.Sa[i,2]-BigM*(1-M.cont_xa[i1,i,2])-BigM*(2-M.w2a[i,m]-w2a[m-1][i1-1]))
                M.rolling_scheda_cons.add(expr=M.Sa[i,2] >= rolling_data.loc[i1-1,'Sa 3']-BigM*(M.cont_xa[i1,i,2])-BigM*(2-M.w2a[i,m]-w2a[m-1][i1-1]))      
            for m in range(1,data['sys'][2]+1):
                #M.rolling_scheda_cons.add(inequality(rolling_data.loc[i-1,'Sa 4']-BigM*(M.cont_xa[i1,i,3])-BigM*(2-M.y[i1,m]-y[m-1][i-1]),M.Sa[i1,3],rolling_data.loc[i-1,'Sa 3']+BigM*(1-M.cont_xa[i1,i,3])+BigM*(2-M.y[i1,m]-y[m-1][i-1])))
                M.rolling_scheda_cons.add(expr=rolling_data.loc[i1-1,'Sa 3'] >= M.Sa[i,3]-BigM*(1-M.cont_xa[i1,i,3])-BigM*(2-M.y[i,m]-y[m-1][i1-1]))
                M.rolling_scheda_cons.add(expr=M.Sa[i,3] >= rolling_data.loc[i1-1,'Sa 4']-BigM*(M.cont_xa[i1,i,3])-BigM*(2-M.y[i,m]-y[m-1][i1-1]))      
            for m in range(1,sum(data['sys'][3])+1):
                #M.rolling_scheda_cons.add(inequality(rolling_data.loc[i-1,'Sb 3']-BigM*(M.cont_xa[i1,i,4])-BigM*(2-M.w4a[i1,m]-w4a[m-1][i-1]),M.Sa[i1,4],rolling_data.loc[i-1,'Sa 4']+BigM*(1-M.cont_xa[i1,i,4])+BigM*(2-M.w4a[i1,m]-w4a[m-1][i-1])))
                M.rolling_scheda_cons.add(expr=rolling_data.loc[i1-1,'Sa 4'] >= M.Sa[i,4]-BigM*(1-M.cont_xa[i1,i,4])-BigM*(2-M.w4a[i,m]-w4a[m-1][i1-1]))
                M.rolling_scheda_cons.add(expr=M.Sa[i,4] >= rolling_data.loc[i1-1,'Sb 3']-BigM*(M.cont_xa[i1,i,4])-BigM*(2-M.w4a[i,m]-w4a[m-1][i1-1]))      
    
    M.rolling_schedb_cons = ConstraintList()
    for i in range(1,nrakes+1):
        for i1 in range(1,cont_rakes+1):
            for m in range(1,data['sys'][2]+1):
                #M.rolling_schedb_cons.add(inequality(rolling_data.loc[i-1,'Sb 2']-BigM*(M.cont_xb[i1,i,3])-BigM*(2-M.y[i1,m]-y[m-1][i1-1]),M.Sb[i1,3],rolling_data.loc[i-1,'Sb 3']+BigM*(1-M.cont_xb[i1,i,3])+BigM*(2-M.y[i1,m]-y[m-1][i1-1])))
                M.schedb_cons.add(expr=rolling_data.loc[i1-1,'Sb 3'] >= M.Sb[i,3]-BigM*(1-M.cont_xb[i1,i,3])-BigM*(2-M.y[i,m]-y[m-1][i1-1]))
                M.schedb_cons.add(expr=M.Sb[i,3] >= rolling_data.loc[i1-1,'Sb 3']-BigM*(M.cont_xb[i1,i,3])-BigM*(2-M.y[i,m]-y[m-1][i1-1]))
            for m in range(1,data['sys'][1]+1):
                #M.rolling_schedb_cons.add(inequality(rolling_data.loc[i-1,'Sb 1']-BigM*(M.cont_xb[i1,i,2])-BigM*(2-M.w2b[i1,m]-w2b[m-1][i1-1]),M.Sb[i1,2],rolling_data.loc[i-1,'Sb 2']+BigM*(1-M.cont_xb[i1,i,2])+BigM*(2-M.w2b[i1,m]-w2b[m-1][i1-1])))
                M.schedb_cons.add(expr=rolling_data.loc[i1-1,'Sb 2'] >= M.Sb[i,2]-BigM*(1-M.cont_xb[i1,i,2])-BigM*(2-M.w2b[i1,m]-w2b[m-1][i1-1]))
                M.schedb_cons.add(expr=M.Sb[i,2] >= rolling_data.loc[i1-1,'Sb 2']-BigM*(M.cont_xb[i1,i,2])-BigM*(2-M.w2b[i1,m]-w2b[m-1][i1-1]))
            for m in range(1,data['sys'][0]+1):
                #M.rolling_schedb_cons.add(inequality(rolling_data.loc[i-1,'Sb 1']+data['t1']-BigM*(M.cont_xb[i1,i,1])-BigM*(2-M.w1b[i1,m]-w1b[m-1][i1-1]),M.Sb[i1,1],rolling_data.loc[i-1,'Sb 1']+BigM*(1-M.cont_xb[i1,i,1])+BigM*(2-M.w1b[i1,m]-w1b[m-1][i1-1])))
                M.schedb_cons.add(expr=rolling_data.loc[i1-1,'Sb 1'] >= M.Sb[i,1]-BigM*(1-M.cont_xb[i1,i,1])-BigM*(2-M.w1b[i1,m]-w1b[m-1][i1-1]))
                M.schedb_cons.add(expr= M.Sb[i,1] >= rolling_data.loc[i1-1,'Sb 1']-BigM*(M.cont_xb[i1,i,1])-BigM*(2-M.w1b[i1,m]-w1b[m-1][i1-1]))
    return M

In [6]:
def get_data(day):
    data = {'pendency':[],  # number of terminals x destinations - only 
        'b':[],   # bpc status of rake - 1 if a rake has to go to that destination (rakes x dest)
        'sys':[2,3,2,[3,3]],
        'a':[],
        'l':[],
        'origin':[],
        'R':[0,90],
        'Pure min limit': 89,
        'Pure max limit': 90,
        'Loading times': [5.0,10.0],
        'cont cost': 6000.0,
        'tau':0.25,
        't1':[0.3,0.5],
        't2':1.0,
        't3':0.5,
        'c1':-13250.0,
        'c2':1.0,
        'c3':6000.0,
        'BigM':200.0,
        'BigM2':100.0}
    if len(data['t1']) != data['sys'][0]:
        print("Error: the size of times and the number of ICP is not same")
        return None
    nrakes = 10
    ndest = 6
    nterm = 2
    l = np.array([[1,0],[1,0],[1,0],[0,1],[0,1],[0,1]])
    data['a'] = [0.0 for _ in range(nrakes)]
    t = 0.0
#     for i in range(len(data['a'])):
#         t += np.random.uniform(low=0.5,high=4.0)
#         data['a'][i] = t+day*24.0
    for i in range(len(data['a'])):
        data['a'][i] = np.random.uniform(low=0.0,high=24.0)+day*24.0
    bpc = np.array([[0.0 for d in range(ndest)]for i in range(nrakes)])
    bpc[4,4] = 1.0
    origin = [sample(list(range(1,data['sys'][0]+1)),1)[0] for _ in range(nrakes)]
    #origin = [2, 1, 2, 2, 1, 2, 1, 1, 1, 2]
    data['b'] = bpc
    data['origin'] = origin
    data['l'] = l
    p = pd.read_csv('data/d.csv',header=None).to_numpy()
    data['pendency'] = p
    return data

In [7]:
def solveModel(M_rolling,data,fname,tlimit = 1800):
    opt = SolverFactory('cplex')
    opt.options['timelimit'] = tlimit
    result_rolling = opt.solve(M_rolling,tee=True)
    print("Solver status :",result_rolling.solver.status)
    print("Termination condition :",result_rolling.solver.termination_condition)
    rolling_result_dataframe = get_resultdataframe(data,M_rolling,fname=fname)
    return rolling_result_dataframe

In [8]:
ndays = 7
results_data = []
datas = []
models = []
reports = []
tlim = 10
np.random.seed(0)
for d in range(ndays):
    print("###################################")
    print("Day -",d+1)
    data = get_data(d)
    if d > 0:
        data['pendency'] += np.random.randint(low=0,high=5,size=(6,2))*20
    datas.append(data)
    if d == 0:
        M = create_rollingModel(None,data,rolling=False)
        results_data.append(solveModel(M,data,'dummy_result_day'+str(d+1),tlimit=tlim))
        models.append(M)
    else:
        rolling_data = get_specTrains(results_data[-1],d+1)
        M = create_rollingModel(rolling_data,data,rolling=True)
        results_data.append(solveModel(M,data,'dummy_result_day'+str(d+1),tlimit=tlim))
        models.append(M)
    reports.append(model_size.build_model_size_report(M))
    print("###################################")

###################################
Day - 1

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\91824\AppData\Local\Temp\tmpf6urrs0v.cplex.log' open.
CPLEX> New value for time limit in seconds: 10
CPLEX> Problem 'C:\Users\91824\AppData\Local\Temp\tmprj9eft54.pyomo.lp' read.
Read time = 0.02 sec. (0.27 ticks)
CPLEX> Problem name         : C:\Users\91824\AppData\Local\Temp\tmprj9eft54.pyomo.lp
Objective sense      : Maximize
Variables            :     796  [Nneg: 91,  Binary: 565,  General Integer: 140]
Objective nonzeros   :     131
Linear constraints   :    2543  [Less: 2262,  Greater: 190,  Equal: 91]
  Nonzeros           :   10811
  

Root relaxation solution time = 0.06 sec. (12.76 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                      -1.24481e+07   4.21977e+07           438.99%
*     0+    0                      -1.06901e+07   4.21977e+07           494.73%
*     0+    0                      -1.06721e+07   4.21977e+07           495.40%
*     0+    0                      -1.06541e+07   4.21977e+07           496.07%
*     0+    0                      -1.06361e+07   4.21977e+07           496.74%
*     0+    0                      -1.06181e+07   4.21977e+07           497.41%
      0     0 -1110350.0000   114  -1.06181e+07 -1110350.0000      424   89.54%
      0     0 -1110350.0000   117  -1.06181e+07     Cuts: 155      738   89.54%
*     0+    0                      -1.00548e+07 -1110350.0000            88.96%
*     0+    0                      -1.00368e+07 -1110350.0000            88.94%
   

*     0+    0                     -7951547.3515 -1113000.0000            86.00%
*     0+    0                     -7915547.3521 -1113000.0000            85.94%
      0     0 -1113015.4403   126 -7915547.3521      Cuts: 24      539   85.94%
      0     0 -1117863.3518   103 -7915547.3521      Cuts: 75      836   85.88%
      0     0 -1117863.3518   115 -7915547.3521      Cuts: 29     1031   85.88%
      0     0 -1580545.9428    98 -7915547.3521      Cuts: 27     1282   80.03%
      0     0 -2190089.4320    96 -7915547.3521      Cuts: 44     1377   72.33%
      0     0 -2719863.3518    44 -7915547.3521      Cuts: 31     1481   65.64%
      0     0 -2719863.3518    81 -7915547.3521      Cuts: 23     1573   65.64%
*     0+    0                     -4225360.1901 -2719863.3518            35.63%
*     0+    0                     -4135579.0064 -2719863.3518            34.23%
*     0+    0                     -3817579.0064 -2719863.3518            28.75%
      0     0        cutoff       -38175

*     0+    0                     -8327086.0039 -1181949.0741            85.81%
*     0+    0                     -8291086.0044 -1181949.0741            85.74%
      0     0 -1181949.0741   103 -8291086.0044      Cuts: 53      795   85.74%
      0     0 -1293936.1111   115 -8291086.0044     Cuts: 101     1056   84.39%
      0     0 -1384065.7407   104 -8291086.0044      Cuts: 44     1243   83.31%
      0     0 -1384065.7407   115 -8291086.0044      Cuts: 53     1352   83.31%
      0     0 -1521743.4833    91 -8291086.0044      Cuts: 12     1448   81.65%
      0     0 -1570805.7099    84 -8291086.0044      Cuts: 25     1568   81.05%
      0     0 -1932014.8148    76 -8291086.0044      Cuts: 31     1672   76.70%
      0     0 -1932014.8148    92 -8291086.0044      Cuts: 22     1748   76.70%
*     0+    0                     -3538102.5926 -1932014.8148            45.39%
*     0+    0                     -3024418.3459 -1932014.8148            36.12%
*     0+    0                     -30176

      0     0 -1545460.7205    88 -9053109.2356      Cuts: 35      979   82.93%
      0     0 -2250894.9310    90 -9053109.2356      Cuts: 29     1134   75.14%
      0     0 -3213144.9310    91 -9053109.2356      Cuts: 27     1183   64.51%
      0     0 -3687144.9310    75 -9053109.2356      Cuts: 17     1232   59.27%
      0     0 -3687144.9310    80 -9053109.2356      Cuts: 17     1282   59.27%
      0     0 -3687144.9310    24 -9053109.2356       Cuts: 6     1295   59.27%
*     0+    0                     -9031909.2349 -3687144.9310            59.18%
*     0+    0                     -8989909.2356 -3687144.9310            58.99%
      0     0 -3687144.9310    50 -8989909.2356      Cuts: 13     1323   58.99%
*     0+    0                     -3922015.9261 -3687144.9310             5.99%
*     0+    0                     -3904757.5667 -3687144.9310             5.57%
      0     2 -3687144.9310     6 -3904757.5667 -3687144.9310     1323    5.57%
Elapsed time = 0.97 sec. (270.11 ticks, 

      0     0 -1242245.2792    99 -8089783.0686      Cuts: 40     1074   84.64%
      0     0 -1327502.8549    69 -8089783.0686      Cuts: 25     1182   83.59%
*     0+    0                     -3237641.2403 -1327502.8549            59.00%
      0     0 -1691965.7895    77 -3237641.2403      Cuts: 27     1397   47.74%
      0     0 -2198998.4646    88 -3237641.2403      Cuts: 27     1474   32.08%
      0     0 -2477650.0000    50 -3237641.2403      Cuts: 13     1517   23.47%
      0     0 -2477650.0000    47 -3237641.2403      Cuts: 14     1545   23.47%
*     0+    0                     -2610150.0000 -2477650.0000             5.08%
      0     2 -2477650.0000    12 -2610150.0000 -2477650.0000     1545    5.08%
Elapsed time = 1.31 sec. (317.23 ticks, tree = 0.02 MB, solutions = 8)
   1481   642 -2558591.9449     7 -2610150.0000 -2477650.0000    12786    5.08%
   3549  1420 -2477650.0000    24 -2610150.0000 -2477650.0000    25732    5.08%

Performing restart 1

Repeating presolve.
Tried 

*   337+  206                     -2414562.7962 -1882849.1038            22.02%
*   426   271      integral     0 -2397367.0823 -1882849.1038     5271   21.46%
*  1212+  627                     -2394099.1038 -1882849.1038            21.35%
*  1304+  744                     -2394099.1038 -1882849.1038            21.35%
   1406   893 -2239862.9927    23 -2394099.1038 -1882849.1038    12673   21.35%
   3320  2141 -2309599.1038    14 -2394099.1038 -1882849.1038    29959   21.35%

Performing restart 1

Repeating presolve.
Tried aggregator 1 time.
MIP Presolve eliminated 137 rows and 30 columns.
MIP Presolve modified 7105 coefficients.
Reduced MIP has 1946 rows, 678 columns, and 8962 nonzeros.
Reduced MIP has 488 binaries, 130 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.22 ticks)
Tried aggregator 1 time.
MIP Presolve modified 210 coefficients.
Reduced MIP has 1946 rows, 678 columns, and 8962 nonzeros.
Reduced MIP has 488 binaries, 130 generals, 0 SOSs, and 0 indicators.


In [9]:
# %run plotting.ipynb  
# for i in range(ndays):
#     print("Day -",i+1)
#     plot_gantt(results_data[i],datas[i],i+1)

In [10]:
for i in range(ndays):
    print(datas[i]['pendency'])

[[ 76  39]
 [ 24  36]
 [ 75  32]
 [ 63  55]
 [ 52  83]
 [ 50 162]]
[[116  99]
 [ 24  56]
 [135  92]
 [123  55]
 [ 72 103]
 [ 70 162]]
[[156  59]
 [ 64  76]
 [ 75  52]
 [ 83  75]
 [ 72 143]
 [110 202]]
[[ 76 119]
 [ 84  76]
 [ 75  52]
 [ 83 115]
 [ 52  83]
 [ 70 202]]
[[136  79]
 [ 44  76]
 [135  92]
 [123  95]
 [112 163]
 [ 70 202]]
[[116  99]
 [ 24 116]
 [ 75  32]
 [103 115]
 [ 92 143]
 [ 50 162]]
[[ 96  99]
 [ 24  36]
 [ 75  52]
 [ 63 135]
 [ 72 143]
 [ 70 162]]
